In [1]:
import h5py
import tensorflow as tf 
import numpy as np

### Parameters

In [2]:
learning_rate = 1e-3
epoch         = 5
batch_size    = 200
save_path     = './model_ckpt/model.ckpt'
filename      = './60000'

### Prepare MNIST data

In [3]:
def readImages_hdf5(filename):
    '''Reads hdf5 file.
       Parameter
       ---------
       filename : the name of the hdf5 file
    '''
    file = h5py.File( filename + '.h5', "r+") #open the hdf5 file.
    
    hdf5_images = np.array(file["/images"]).astype("uint8") #read the images as np array
    hdf5_labels = np.array(file["/meta"]).astype("uint8")
    
    return hdf5_images, hdf5_labels

In [4]:
def one_hot_encoder(label_arr):
    '''Returns the given MNIST labels from np arrays of integers to np array of one hot labels.
       Parameter
       ---------
       label_arr : np array of MNIST integer labels
    '''
    total_labels  = label_arr.shape[0] #get the total number of labels
    one_hot_label = np.zeros([total_labels, 10]) #10 for num of classes in MNIST
    
    for i in range(label_arr.shape[0]): #loop through all the labels
        
        one_hot_label[i][int(label_arr[i])] = 1.0 #the label value will be marked as 1.0 at that specific index
        
    return one_hot_label #returns the np one-hot label 

In [5]:
images, labels = readImages_hdf5(filename)
labels = one_hot_encoder(labels)
images = images.reshape(images.shape[0],28, 28, 1) #reshape into a tensor of rank 4 for CNN filtering

### Graph

In [6]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

class Model():
           
    
    def __init__(self):
    
        self.x = tf.placeholder(tf.float32, [None, 28,28,1], name='inputs')
        self.y = tf.placeholder(tf.float32, [None, 10], name='targets')
        self.is_train = tf.placeholder(tf.bool, name='is_train')

        conv1 = tf.contrib.layers.conv2d(self.x, num_outputs=64, kernel_size=3, stride=1,padding='SAME', activation_fn=None)
    
        #batch normalization before the activation function but after the linear transformation.
        conv1_norm = tf.layers.batch_normalization(conv1, training=self.is_train)
        conv1_actv = tf.nn.relu(conv1_norm)

        conv2 = tf.contrib.layers.conv2d(conv1_actv, num_outputs=64, kernel_size=3, stride=2,padding='SAME', activation_fn=None)
        
        #batch normalization before the activation function but after the linear transformation.
        conv2_norm = tf.layers.batch_normalization(conv2, training=self.is_train)
        
        conv2_actv = tf.nn.relu(conv2_norm)
        

        output_size = 14*14*64
        output_layer = tf.reshape(conv2_actv, (-1, output_size))


        W2 = tf.Variable(tf.truncated_normal([output_size, 100], stddev=0.1))
        B2 = tf.Variable(tf.ones([100]))

        fc2 =  tf.add(tf.matmul(output_layer, W2), B2)
        
        #batch normalization before the activation function but after the linear transformation.
        fc2_norm = tf.layers.batch_normalization(fc2, training=self.is_train)
        fc2_actv = tf.nn.relu(fc2_norm)

        W3 = tf.Variable(tf.truncated_normal([100, 10], stddev=0.1))
        B3 = tf.Variable(tf.ones([10]))

        self.logits = tf.add(tf.matmul(fc2_actv, W3), B3)

        Y_pred = tf.nn.softmax(self.logits)
    
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y, logits=self.logits))
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS) #IMPORTANT
        with tf.control_dependencies(update_ops): #IMPORTANT
            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)
        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

### Building the graph for training

In [7]:
model_cnn = Model()

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())

try:

    saver.restore(sess, save_path)
    print("Model has been loaded !")

except:

    print("Model is not loaded !")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:Restoring parameters from ./model_ckpt/model.ckpt
Model is not loaded !


### Training

In [8]:
for epoch_idx in range(epoch):
    
    loss_total = 0
    accuracy_total = 0
    counter = 0
    
    for firstidx in range(0, images.shape[0], batch_size):
        
        finalidx = firstidx + batch_size
        if finalidx >= images.shape[0] : finalidx = images.shape[0] #to prevent the index to be out of range.
        
        _, loss_val, accuracy_val = sess.run([model_cnn.optimizer, model_cnn.loss, model_cnn.accuracy], 
                                              feed_dict={
                                                        model_cnn.x : images[firstidx:finalidx],
                                                        model_cnn.y : labels[firstidx:finalidx],
                                                        model_cnn.is_train : True
                                                        })
        loss_total += loss_val
        accuracy_total += accuracy_val
        counter += 1
        
    print("Epoch %d, loss %g, accuracy %g"%(epoch_idx, loss_total, accuracy_total/counter))
        
    saver.save(sess, save_path)

sess.close()

Epoch 0, loss 64.8019, accuracy 0.948533
Epoch 1, loss 14.0743, accuracy 0.9878
Epoch 2, loss 7.84788, accuracy 0.994167
Epoch 3, loss 4.45371, accuracy 0.997233
Epoch 4, loss 2.38387, accuracy 0.999117


### Testing session 

(Testing with the same trained data because the purpose is just to check if the estimated population mean and population variance works or not.)

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

model_cnn = Model()

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())

try:

    saver.restore(sess, save_path)
    print("Model has been loaded !")

except:

    print("Model is not loaded !")

INFO:tensorflow:Restoring parameters from ./model_ckpt/model.ckpt
Model has been loaded !


In [10]:
counter = 0

for firstidx in range(0, images.shape[0], batch_size):

    finalidx = firstidx + batch_size
    if finalidx >= images.shape[0] : finalidx = images.shape[0] #to prevent the index to be out of range.

    loss_val, accuracy_val = sess.run([model_cnn.loss, model_cnn.accuracy], 
                                          feed_dict={
                                                    model_cnn.x : images[firstidx:finalidx],
                                                    model_cnn.y : labels[firstidx:finalidx],
                                                    model_cnn.is_train : False
                                                    })
    
    counter += 1

print("Loss %g, accuracy %g"%(loss_total, accuracy_total/counter))

sess.close()

Loss 2.38387, accuracy 0.999117
